In [2]:
import cv2
import numpy as np
import mediapipe as mp
from keras.models import load_model

model = load_model('model_alpha.h5')

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

alphabets = [chr(i) for i in range(ord('A'), ord('Z') + 1)]

with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                landmarks = []
                for landmark in hand_landmarks.landmark:
                    landmarks.append(landmark.x)
                    landmarks.append(landmark.y)
                    landmarks.append(landmark.z)
                landmarks = np.array(landmarks)
                landmarks = landmarks.reshape(1, -1)
                landmarks = (landmarks - np.mean(landmarks)) / np.std(landmarks)

                predictions = model.predict(landmarks)
                predicted_index = np.argmax(predictions)
                predicted_gesture = alphabets[predicted_index]

                cv2.putText(frame, f'Predicted: {predicted_gesture}', (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow('Hand Gesture Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 33ms/step
